In [1]:
pip install selenium

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import time
import numpy as np
import pandas as pd
import time
import pickle
import sys
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By

In [ ]:
driver = webdriver.Chrome('./chromedriver')
driver.get('https:/1xbet.whoscored.com/')
driver.implicitly_wait(3)
driver.close()


# Summary

In [14]:
# Premier league 1920 plyaer stat table(URL)
URL2223 = 'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/9075/Stages/20934/PlayerStatistics/England-Premier-League-2022-2023'

In [73]:
def crawling_player_summary(URL):
    """
    crawling player statistics of certain league and season.
    
    Args :
        URL : player statistics webpage URL
        
    return :
        player statistics (dataframe)
    
    """
    url = str(URL)
    driver = webdriver.Chrome('./chromedriver')
    driver.get(url)
    
    time.sleep(3)
    driver.find_element(By.XPATH, '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]').click()
    time.sleep(3)
    driver.set_window_size(1552, 832)
    driver.execute_script("window.scrollTo(0,300)")    
    page = driver.find_elements(By.CSS_SELECTOR, '#statistics-paging-summary > div > dl.listbox.right > dt > b')[0].get_attribute("textContent").split("\\")[0]
    page = int(page.split('/')[1].split(' ')[0])
    player_statistics_df = pd.DataFrame(columns=[
        "Name", "Team", "Age", "Position", "Apps", "Minutes",
        "Total Goals", "Total Assists", "Yellow cards", "Red cards","Shots per game", "Pass success percentage","Aerials won per game","Man of the match", "Rating"])
    
    with tqdm(total=page, file=sys.stdout) as pbar :
        for i in range(page):
            if i != 0:
                driver.find_element(By.ID, "next").click()
                time.sleep(3)

            temp_df = pd.DataFrame(columns=[
        "Name", "Team", "Age", "Position", "Apps", "Minutes",
        "Total Goals", "Total Assists", "Yellow cards", "Red cards","Shots per game", "Pass success percentage","Aerials won per game","Man of the match", "Rating"])
            elements = driver.find_elements(By.CSS_SELECTOR, "#player-table-statistics-body tr")

            for element in elements:
                player_statistics_dict = { 
                    "Name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[0].
                    find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0],
                    "Team": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, ("a"))[1].find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0].split(",")[0],
                    "Age": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "span")[4].get_attribute("textContent").split("\t")[0],
                    "Position": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "span")[5].get_attribute("textContent").split("\t")[0][2:],
                    "Apps": element.find_elements(By.CSS_SELECTOR, "td")[2].get_attribute("textContent").split("\t")[0],
                    "Minutes": element.find_elements(By.CSS_SELECTOR, "td")[3].get_attribute("textContent").split("\t")[0], 
                    "Total Goals": element.find_elements(By.CSS_SELECTOR, "td")[4].get_attribute("textContent").split("\t")[0],
                    "Total Assists": element.find_elements(By.CSS_SELECTOR, "td")[5].get_attribute("textContent").split("\t")[0], 
                    "Yellow cards": element.find_elements(By.CSS_SELECTOR, "td")[6].get_attribute("textContent").split("\t")[0], 
                    "Red cards": element.find_elements(By.CSS_SELECTOR, "td")[7].get_attribute("textContent").split("\t")[0],
                    "Shots per game": element.find_elements(By.CSS_SELECTOR, "td")[8].get_attribute("textContent").split("\t")[0],
                    "Pass success percentage": element.find_elements(By.CSS_SELECTOR, "td")[9].get_attribute("textContent").split("\t")[0],
                    "Aerials won per game": element.find_elements(By.CSS_SELECTOR, "td")[10].get_attribute("textContent").split("\t")[0],
                    "Man of the match": element.find_elements(By.CSS_SELECTOR, "td")[11].get_attribute("textContent").split("\t")[0],
                    "Rating": element.find_elements(By.CSS_SELECTOR, "td")[12].get_attribute("textContent").split("\t")[0],
                }
                temp_df.loc[len(temp_df)] = player_statistics_dict

            player_statistics_df = pd.concat([player_statistics_df,temp_df])
            pbar.update(1)
            pbar.set_description('page {}'.format(i+1))
            
    driver.close()
    player_statistics_df.replace("-", "0", inplace=True)
    return(player_statistics_df)

In [26]:
PremierLeagueSummary = crawling_player_summary(URL2223)

C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\413797012.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 31: 100%|█████████████████████████████████████████████████████████████████████████| 31/31 [04:45<00:00,  9.19s/it]


In [27]:
PremierLeagueSummary.head()

,Name,Team,Age,Position,Apps,Minutes,Total Goals,Total Assists,Yellow cards,Red cards,Shots per game,Pass success percentage,Aerials won per game,Man of the match,Rating
0,Gabriel Jesus,Arsenal,25,"AM(LR),FW",14,1226,5,5,4,0,3.6,81.9,1.7,2,7.61
1,Kieran Trippier,Newcastle,32,"D(LR),M(R)",24,2111,1,4,5,0,0.4,74.6,1.3,7,7.61
2,Erling Haaland,Man City,22,FW,24(1),2030,27,5,4,0,3.4,76.1,1.2,5,7.57
3,Kevin De Bruyne,Man City,31,"M(CLR),FW",22(2),1933,4,12,1,0,2.4,78.8,0.3,7,7.56
4,Bukayo Saka,Arsenal,21,"D(L),M(CLR)",26,2251,10,9,5,0,2.5,81.3,0.3,2,7.45


# Offensive

In [95]:
def crawling_player_offensive(URL):
    """
    crawling player statistics(offensive) of certain league and season.
    
    Args :
        URL : player statistics webpage URL
        
    return :
        player statistics (dataframe)
    
    """
    url = str(URL)
    driver = webdriver.Chrome('./chromedriver')
    driver.get(url)
    driver.find_element(By.XPATH, '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]').click()
    time.sleep(3)
    driver.set_window_size(1552, 832)
    driver.execute_script("window.scrollTo(0,300)")    
    
    time.sleep(3)
    driver.find_element(By.LINK_TEXT, 'next').click()

    time.sleep(2)

    driver.find_element(By.LINK_TEXT, 'first').click()
    
    time.sleep(2.5)
    
    driver.find_element(By.LINK_TEXT, "Offensive").click()
    
    time.sleep(2.5)
    page = driver.find_elements(By.CSS_SELECTOR, '#statistics-paging-offensive > div > dl.listbox.right > dt > b')[0].get_attribute("textContent").split("\t")[0]
    page = int(page.split('/')[1].split(' ')[0])
    
    player_statistics_df = pd.DataFrame(columns=[
            "Name", "Team", "Age", "Position","Apps", "Minutes", "Total goals", "Total Assists",
            "Shots per game", "Key passes per game", "Dribble","Fouled per game", "Offsides per game","Dispossessed per game","Bad control per game","Rating"])
    
    with tqdm(total=page, file=sys.stdout) as pbar :
        for i in range(page):
            if i != 0:
                driver.find_element(By.LINK_TEXT,'next').click()
                time.sleep(2.5)

            temp_df = pd.DataFrame(columns=[
            "Name", "Team", "Age", "Position","Apps", "Minutes", "Total goals", "Total Assists",
            "Shots per game", "Key passes per game", "Dribble","Fouled per game", "Offsides per game","Dispossessed per game","Bad control per game","Rating"])

            elements = driver.find_elements(By.CSS_SELECTOR, "#player-table-statistics-body tr")[10:]

            for element in elements:
                player_statistics_dict = { 
                    "Name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[0].
                    find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0],
                    "Team": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR,"a")[1].find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0].split(",")[0],
                    "Age": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR,"span")[4].get_attribute("textContent").split("\t")[0],
                    "Position": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "span")[5].get_attribute("textContent").split("\t")[0][2:],
                    "Apps": element.find_elements(By.CSS_SELECTOR, "td")[2].get_attribute("textContent").split("\t")[0],
                    "Minutes": element.find_elements(By.CSS_SELECTOR, "td")[3].get_attribute("textContent").split("\t")[0], 
                    "Total goals": element.find_elements(By.CSS_SELECTOR, "td")[4].get_attribute("textContent").split("\t")[0],
                    "Total Assists": element.find_elements(By.CSS_SELECTOR, "td")[5].get_attribute("textContent").split("\t")[0], 
                    "Shots per game": element.find_elements(By.CSS_SELECTOR, "td")[6].get_attribute("textContent").split("\t")[0], 
                    "Key passes per game": element.find_elements(By.CSS_SELECTOR, "td")[7].get_attribute("textContent").split("\t")[0],
                    "Dribble": element.find_elements(By.CSS_SELECTOR, "td")[8].get_attribute("textContent").split("\t")[0],
                    "Fouled per game": element.find_elements(By.CSS_SELECTOR, "td")[9].get_attribute("textContent").split("\t")[0],
                    "Offsides per game": element.find_elements(By.CSS_SELECTOR, "td")[10].get_attribute("textContent").split("\t")[0],
                    "Dispossessed per game": element.find_elements(By.CSS_SELECTOR, "td")[11].get_attribute("textContent").split("\t")[0],
                    "Bad control per game": element.find_elements(By.CSS_SELECTOR, "td")[12].get_attribute("textContent").split("\t")[0],
                    "Rating": element.find_elements(By.CSS_SELECTOR, "td")[13].get_attribute("textContent").split("\t")[0],
                }
                temp_df.loc[len(temp_df)] = player_statistics_dict


            player_statistics_df = pd.concat([player_statistics_df,temp_df])
            pbar.update(1)
            pbar.set_description('page {}'.format(i+1))
        
    driver.close()

    player_statistics_df.replace("-", "0", inplace=True)
    return(player_statistics_df)

In [96]:
PremierLeagueOffensive = crawling_player_offensive(URL2223)

C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\3162353173.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 31: 100%|█████████████████████████████████████████████████████████████████████████| 31/31 [04:08<00:00,  8.03s/it]


In [97]:
PremierLeagueOffensive.head()

,Name,Team,Age,Position,Apps,Minutes,Total goals,Total Assists,Shots per game,Key passes per game,Dribble,Fouled per game,Offsides per game,Dispossessed per game,Bad control per game,Rating
0,Gabriel Jesus,Arsenal,25,"AM(LR),FW",14,1226,5,5,3.6,1.6,2.1,2.7,0.6,2.9,3.9,7.61
1,Kieran Trippier,Newcastle,32,"D(LR),M(R)",24,2111,1,4,0.4,3,0.5,0.9,0.3,0.2,0.7,7.61
2,Erling Haaland,Man City,22,FW,24(1),2030,27,5,3.4,0.8,0.2,0.7,0.2,0.9,1.3,7.57
3,Kevin De Bruyne,Man City,31,"M(CLR),FW",22(2),1933,4,12,2.4,3.3,1,0.8,0.1,1,1.5,7.56
4,Bukayo Saka,Arsenal,21,"D(L),M(CLR)",26,2251,10,9,2.5,2.2,1.7,1.7,0.2,1.6,2.2,7.45


# Defensive

In [75]:
def crawling_player_defensive(URL):
    """
    crawling player statistics(defensive) of certain league and season.
    
    Args :
        URL : player statistics webpage URL
        
    return :
        player statistics (dataframe)
    
    """
    url = str(URL)
    driver = webdriver.Chrome('./chromedriver')
    driver.get(url)
    driver.find_element(By.XPATH, '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]').click()
    time.sleep(3)
    driver.set_window_size(1552, 832)
    driver.execute_script("window.scrollTo(0,300)")   
    
    time.sleep(2.5)

    driver.find_element(By.ID, 'next').click()

    time.sleep(2.5)

    driver.find_element(By.ID, 'first').click()
    
    time.sleep(2.5)
    
    driver.find_element(By.LINK_TEXT, "Defensive").click()
    
    time.sleep(2.5)
    page = driver.find_elements(By.CSS_SELECTOR, '#statistics-paging-defensive > div > dl.listbox.right > dt > b')[0].get_attribute("textContent")
    page = int(page.split('/')[1].split(' ')[0])
    
    player_statistics_df = pd.DataFrame(columns=[
            "Name", "Team", "Age", "Position","Apps", "Minutes", "Tackles per game", "Interceptions per game",
            "Fouls per game", "Offsides per game", "Clearances per game","Dribbled past per game", "Outfield blocks per game","Own goals","Rating"])
    
    with tqdm(total=page, file=sys.stdout) as pbar :
        for i in range(page):
            if i != 0:
                driver.find_element(By.LINK_TEXT, 'next').click()
                time.sleep(2.5)

            temp_df = pd.DataFrame(columns=[
            "Name", "Team", "Age", "Position","Apps", "Minutes", "Tackles per game", "Interceptions per game",
            "Fouls per game", "Offsides per game", "Clearances per game","Dribbled past per game", "Outfield blocks per game","Own goals","Rating"])

            elements = driver.find_elements(By.CSS_SELECTOR, "#player-table-statistics-body tr")[10:]

            for element in elements:
                player_statistics_dict = { 
                    "Name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[0].find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0],
                    "Team": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[1].find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0].split(",")[0],
                    "Age": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "span")[4].get_attribute("textContent").split("\t")[0],
                    "Position": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "span")[5].get_attribute("textContent").split("\t")[0][2:],
                    "Apps": element.find_elements(By.CSS_SELECTOR, "td")[2].get_attribute("textContent").split("\t")[0],
                    "Minutes": element.find_elements(By.CSS_SELECTOR, "td")[3].get_attribute("textContent").split("\t")[0], 
                    "Tackles per game": element.find_elements(By.CSS_SELECTOR, "td")[4].get_attribute("textContent").split("\t")[0],
                    "Interceptions per game": element.find_elements(By.CSS_SELECTOR, "td")[5].get_attribute("textContent").split("\t")[0], 
                    "Fouls per game": element.find_elements(By.CSS_SELECTOR, "td")[6].get_attribute("textContent").split("\t")[0], 
                    "Offsides per game": element.find_elements(By.CSS_SELECTOR, "td")[7].get_attribute("textContent").split("\t")[0],
                    "Clearances per game": element.find_elements(By.CSS_SELECTOR, "td")[8].get_attribute("textContent").split("\t")[0],
                    "Dribbled past per game": element.find_elements(By.CSS_SELECTOR, "td")[9].get_attribute("textContent").split("\t")[0],
                    "Outfield blocks per game": element.find_elements(By.CSS_SELECTOR, "td")[10].get_attribute("textContent").split("\t")[0],
                    "Own goals": element.find_elements(By.CSS_SELECTOR, "td")[11].get_attribute("textContent").split("\t")[0],
                    "Rating": element.find_elements(By.CSS_SELECTOR, "td")[12].get_attribute("textContent").split("\t")[0],
                }
                temp_df.loc[len(temp_df)] = player_statistics_dict


            player_statistics_df = pd.concat([player_statistics_df,temp_df])
            pbar.update(1)
            pbar.set_description('page {}'.format(i+1))
        
    driver.close()
    
    player_statistics_df.replace("-", "0", inplace=True)
    return(player_statistics_df)

In [48]:
PremierLeagueDefensive = crawling_player_defensive(URL2223)

C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\2379465608.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 31: 100%|█████████████████████████████████████████████████████████████████████████| 31/31 [03:58<00:00,  7.71s/it]


In [49]:
PremierLeagueDefensive.head()

,Name,Team,Age,Position,Apps,Minutes,Tackles per game,Interceptions per game,Fouls per game,Offsides per game,Clearances per game,Dribbled past per game,Outfield blocks per game,Own goals,Rating
0,Gabriel Jesus,Arsenal,25,"AM(LR),FW",14,1226,1.4,0.6,1.7,0,0.6,0.8,0.1,0,7.61
1,Kieran Trippier,Newcastle,32,"D(LR),M(R)",24,2111,2,1.3,1.2,0.3,1.8,1.1,0.5,0,7.61
2,Erling Haaland,Man City,22,FW,24(1),2030,0.1,0.1,0.9,0,0.4,0.1,0.1,0,7.57
3,Kevin De Bruyne,Man City,31,"M(CLR),FW",22(2),1933,0.9,0.2,0.8,0,0.4,0.9,0.1,0,7.56
4,Bukayo Saka,Arsenal,21,"D(L),M(CLR)",26,2251,1.4,0.5,1.1,0.1,0.5,0.5,0,0,7.45


# Passing

In [76]:
def crawling_player_passing(URL):
    url = str(URL)
    driver = webdriver.Chrome('./chromedriver')
    driver.get(url)
    driver.find_element(By.XPATH, '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]').click()
    time.sleep(3)
    driver.set_window_size(1552, 832)
    driver.execute_script("window.scrollTo(0,300)")   
    time.sleep(2.5)

    driver.find_element(By.LINK_TEXT, 'next').click()

    time.sleep(2.5)

    driver.find_element(By.LINK_TEXT, 'first').click()
    
    time.sleep(2.5)
    
    driver.find_element(By.LINK_TEXT, "Passing").click()
    
    time.sleep(2.5)
    page = driver.find_elements(By.CSS_SELECTOR, '#statistics-paging-passing > div > dl.listbox.right > dt > b')[0].get_attribute("textContent")
    page = int(page.split('/')[1].split(' ')[0])
    
    player_statistics_df = pd.DataFrame(columns=[
            "Name", "Team", "Age", "Position","Apps", "Minutes", "Total Assists", "Key passes per game",
            "Passes per game", "Pass success percentage","Crosses per game", "Long balls per game","Through balls per game","Rating"])
    
    with tqdm(total=page, file=sys.stdout) as pbar :
        for i in range(page):
            if i != 0:
                driver.find_element(By.LINK_TEXT, 'next').click()
                time.sleep(2.5)

            temp_df = pd.DataFrame(columns=[
            "Name", "Team", "Age", "Position","Apps", "Minutes", "Total Assists", "Key passes per game",
            "Passes per game", "Pass success percentage","Crosses per game", "Long balls per game","Through balls per game","Rating"])

            elements = driver.find_elements(By.CSS_SELECTOR, "#player-table-statistics-body tr")[10:]

            for element in elements:
                player_statistics_dict = { 
                    "Name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[0].
                    find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0],
                    "Team": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[1].find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0].split(",")[0],
                    "Age": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "span")[4].get_attribute("textContent").split("\t")[0],
                    "Position": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "span")[5].get_attribute("textContent").split("\t")[0][2:],
                    "Apps": element.find_elements(By.CSS_SELECTOR, "td")[2].get_attribute("textContent").split("\t")[0],
                    "Minutes": element.find_elements(By.CSS_SELECTOR, "td")[3].get_attribute("textContent").split("\t")[0], 
                    "Total Assists": element.find_elements(By.CSS_SELECTOR, "td")[4].get_attribute("textContent").split("\t")[0],
                    "Key passes per game": element.find_elements(By.CSS_SELECTOR, "td")[5].get_attribute("textContent").split("\t")[0], 
                    "Passes per game": element.find_elements(By.CSS_SELECTOR, "td")[6].get_attribute("textContent").split("\t")[0], 
                    "Pass success percentage": element.find_elements(By.CSS_SELECTOR, "td")[7].get_attribute("textContent").split("\t")[0],
                    "Crosses per game": element.find_elements(By.CSS_SELECTOR, "td")[8].get_attribute("textContent").split("\t")[0],
                    "Long balls per game": element.find_elements(By.CSS_SELECTOR, "td")[9].get_attribute("textContent").split("\t")[0],
                    "Through balls per game": element.find_elements(By.CSS_SELECTOR, "td")[10].get_attribute("textContent").split("\t")[0],
                    "Rating": element.find_elements(By.CSS_SELECTOR, "td")[11].get_attribute("textContent").split("\t")[0],
                }
                temp_df.loc[len(temp_df)] = player_statistics_dict


            player_statistics_df = pd.concat([player_statistics_df,temp_df])
            pbar.update(1)
            pbar.set_description('page {}'.format(i+1))
        
    driver.close()
    player_statistics_df.replace("-", "0", inplace=True)

    return(player_statistics_df)

In [45]:
PremierLeaguePassing = crawling_player_passing(URL2223)

C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\1288772440.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 31: 100%|█████████████████████████████████████████████████████████████████████████| 31/31 [03:36<00:00,  6.98s/it]


In [46]:
PremierLeaguePassing.head()

,Name,Team,Age,Position,Apps,Minutes,Total Assists,Key passes per game,Passes per game,Pass success percentage,Crosses per game,Long balls per game,Through balls per game,Rating
0,Gabriel Jesus,Arsenal,25,"AM(LR),FW",14,1226,5,1.6,27.6,81.9,0.1,0.5,0.1,7.61
1,Kieran Trippier,Newcastle,32,"D(LR),M(R)",24,2111,4,3,51.5,74.6,3.9,3.4,0.3,7.61
2,Erling Haaland,Man City,22,FW,24(1),2030,5,0.8,13.9,76.1,0,0,0.1,7.57
3,Kevin De Bruyne,Man City,31,"M(CLR),FW",22(2),1933,12,3.3,42.3,78.8,2.4,2.5,0.4,7.56
4,Bukayo Saka,Arsenal,21,"D(L),M(CLR)",26,2251,9,2.2,34.2,81.3,1,1,0,7.45


# To CSV

In [77]:
import pandas as pd

def Download(URL, name, year):
    # Appel des fonctions de crawling pour récupérer les statistiques des joueurs
    stats_summary = crawling_player_summary(URL)
    print('player stats summary : done')
    stats_defensive = crawling_player_defensive(URL)
    print('player stats defensive : done')
    stats_offensive = crawling_player_offensive(URL)
    print('player stats offensive : done')
    stats_passing = crawling_player_passing(URL)
    print('player stats passing : done')
    
    # Sauvegarde des données dans des fichiers CSV
    summary_file_name = name + '-Summary-' + year + '.csv'
    stats_summary_df = pd.DataFrame(stats_summary)
    stats_summary_df.to_csv(summary_file_name, index=False)
    print(f'player stats summary saved to {summary_file_name}')
    
    defensive_file_name = name + '-Defensive-' + year + '.csv'
    stats_defensive_df = pd.DataFrame(stats_defensive)
    stats_defensive_df.to_csv(defensive_file_name, index=False)
    print(f'player stats defensive saved to {defensive_file_name}')
    
    offensive_file_name = name + '-Offensive-' + year + '.csv'
    stats_offensive_df = pd.DataFrame(stats_offensive)
    stats_offensive_df.to_csv(offensive_file_name, index=False)
    print(f'player stats offensive saved to {offensive_file_name}')
    
    passing_file_name = name + '-Passing-' + year + '.csv'
    stats_passing_df = pd.DataFrame(stats_passing)
    stats_passing_df.to_csv(passing_file_name, index=False)
    print(f'player stats passing saved to {passing_file_name}')
    
    # Retourne le téléchargement de chacun des CSV
    return (summary_file_name, defensive_file_name, offensive_file_name, passing_file_name)

In [98]:
import pandas as pd

def Download(URL, name, year):
    # Appel des fonctions de crawling pour récupérer les statistiques des joueurs
    stats_offensive = crawling_player_offensive(URL)
    print('player stats offensive : done')
    
    offensive_file_name = name + '-Offensive-' + year + '.csv'
    stats_offensive_df = pd.DataFrame(stats_offensive)
    stats_offensive_df.to_csv(offensive_file_name, index=False)
    print(f'player stats offensive saved to {offensive_file_name}')
    
    # Retourne le téléchargement de chacun des CSV
    return (offensive_file_name)

In [99]:
PL1920 = Download(URLPL1920, 'PremierLeague', '2019-2020')
PL2021 = Download(URLPL2021, 'PremierLeague', '2020-2021')

C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\3162353173.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 30: 100%|█████████████████████████████████████████████████████████████████████████| 30/30 [04:10<00:00,  8.34s/it]
player stats offensive : done
player stats offensive saved to PremierLeague-Offensive-2019-2020.csv


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\3162353173.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 30: 100%|█████████████████████████████████████████████████████████████████████████| 30/30 [03:29<00:00,  6.99s/it]
player stats offensive : done
player stats offensive saved to PremierLeague-Offensive-2020-2021.csv


In [100]:
PL2122 = Download(URLPL2122, 'PremierLeague', '2021-2022')
PL2223 = Download(URLPL2223, 'PremierLeague', '2022-2023')

C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\3162353173.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 30: 100%|█████████████████████████████████████████████████████████████████████████| 30/30 [03:30<00:00,  7.03s/it]
player stats offensive : done
player stats offensive saved to PremierLeague-Offensive-2021-2022.csv


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\3162353173.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 31: 100%|█████████████████████████████████████████████████████████████████████████| 31/31 [03:45<00:00,  7.29s/it]
player stats offensive : done
player stats offensive saved to PremierLeague-Offensive-2022-2023.csv


In [101]:
BD1920 = Download(URLBD1920, 'Bundesliga', '2019-2020')
BD2021 = Download(URLBD2021, 'Bundesliga','2019-2020')
BD2122 = Download(URLBD2122, 'Bundesliga', '2020-2021')
BD2223 = Download(URLBD2223, 'Bundesliga', '2022-2023')

C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\3162353173.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 27: 100%|█████████████████████████████████████████████████████████████████████████| 27/27 [03:25<00:00,  7.61s/it]
player stats offensive : done
player stats offensive saved to Bundesliga-Offensive-2019-2020.csv


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\3162353173.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 29: 100%|█████████████████████████████████████████████████████████████████████████| 29/29 [03:37<00:00,  7.50s/it]
player stats offensive : done
player stats offensive saved to Bundesliga-Offensive-2019-2020.csv


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\3162353173.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 30: 100%|█████████████████████████████████████████████████████████████████████████| 30/30 [03:40<00:00,  7.34s/it]
player stats offensive : done
player stats offensive saved to Bundesliga-Offensive-2020-2021.csv


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\3162353173.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 29: 100%|█████████████████████████████████████████████████████████████████████████| 29/29 [03:26<00:00,  7.12s/it]
player stats offensive : done
player stats offensive saved to Bundesliga-Offensive-2022-2023.csv


In [102]:
SA1920 = Download(URLSA1920, 'SerieA', '2019-2020')
SA2021 = Download(URLSA2021, 'SerieA', '2020-2021')
SA2122 = Download(URLSA2122, 'SerieA', '2020-2021')
SA2223 = Download(URLSA2223, 'SerieA', '2022-2023')

C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\3162353173.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 31: 100%|█████████████████████████████████████████████████████████████████████████| 31/31 [03:49<00:00,  7.41s/it]
player stats offensive : done
player stats offensive saved to SerieA-Offensive-2019-2020.csv


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\3162353173.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 35: 100%|█████████████████████████████████████████████████████████████████████████| 35/35 [04:27<00:00,  7.63s/it]
player stats offensive : done
player stats offensive saved to SerieA-Offensive-2020-2021.csv


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\3162353173.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 34: 100%|█████████████████████████████████████████████████████████████████████████| 34/34 [04:08<00:00,  7.30s/it]
player stats offensive : done
player stats offensive saved to SerieA-Offensive-2020-2021.csv


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\3162353173.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 34: 100%|█████████████████████████████████████████████████████████████████████████| 34/34 [04:03<00:00,  7.17s/it]
player stats offensive : done
player stats offensive saved to SerieA-Offensive-2022-2023.csv


In [104]:
LONE1920 = Download(URLLONE1920, 'Ligue1', '2019-2020')
LONE2021 = Download(URLLONE2021, 'Ligue1', '2019-2020')
LONE2122 = Download(URLLONE2122, 'Ligue1', '2020-2021')
LONE2223 = Download(URLLONE2223, 'Ligue1', '2022-2023')

C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\3162353173.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 31: 100%|█████████████████████████████████████████████████████████████████████████| 31/31 [03:42<00:00,  7.18s/it]
player stats offensive : done
player stats offensive saved to Ligue1-Offensive-2019-2020.csv


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\3162353173.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 34: 100%|█████████████████████████████████████████████████████████████████████████| 34/34 [04:15<00:00,  7.52s/it]
player stats offensive : done
player stats offensive saved to Ligue1-Offensive-2019-2020.csv


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\3162353173.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 31: 100%|█████████████████████████████████████████████████████████████████████████| 31/31 [03:47<00:00,  7.35s/it]
player stats offensive : done
player stats offensive saved to Ligue1-Offensive-2020-2021.csv


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\3162353173.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 32: 100%|█████████████████████████████████████████████████████████████████████████| 32/32 [04:13<00:00,  7.92s/it]
player stats offensive : done
player stats offensive saved to Ligue1-Offensive-2022-2023.csv


In [ ]:
LIGA1920 = Download(URLLIGA1920, 'LaLiga','2019-2020')
LIGA2021 = Download(URLLIGA2021, 'LaLiga','2019-2020')
LIGA2122 = Download(URLLIGA2122, 'LaLiga', '2020-2021')
LIGA2223 = Download(URLLIGA2223, 'LaLiga', '2022-2023')

C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\3162353173.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 16:  53%|██████████████████████████████████████▉                                  | 16/30 [01:51<01:48,  7.76s/it]

In [87]:
URLPL1920 = 'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/8618/Stages/12496/PlayerStatistics/England-Premier-League-2019-2020'
URLPL2021 = 'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/8228/Stages/18685/PlayerStatistics/England-Premier-League-2020-2021'
URLPL2122 = 'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/8618/Stages/19793/PlayerStatistics/England-Premier-League-2021-2022'
URLPL2223 = 'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/9075/Stages/20934/PlayerStatistics/England-Premier-League-2022-2023'

In [88]:
PL1920 = Download(URLPL1920, 'PremierLeague', '2019-2020')
PL2021 = Download(URLPL2021, 'PremierLeague', '2020-2021')

C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\4206667366.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 30: 100%|█████████████████████████████████████████████████████████████████████████| 30/30 [03:46<00:00,  7.54s/it]
player stats summary : done


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\2573841616.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 30: 100%|█████████████████████████████████████████████████████████████████████████| 30/30 [03:31<00:00,  7.06s/it]
player stats defensive : done


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\2937695269.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


...

page 30: 100%|█████████████████████████████████████████████████████████████████████████| 30/30 [03:27<00:00,  6.90s/it]
player stats defensive : done


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\2937695269.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 30: 100%|█████████████████████████████████████████████████████████████████████████| 30/30 [03:43<00:00,  7.46s/it]
player stats offensive : done


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\167999107.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]"}
  (Session info: chrome=110.0.5481.177)
Stacktrace:
Backtrace:
	(No symbol) [0x00E837D3]
	(No symbol) [0x00E18B81]
	(No symbol) [0x00D1B36D]
	(No symbol) [0x00D4D382]
	(No symbol) [0x00D4D4BB]
	(No symbol) [0x00D83302]
	(No symbol) [0x00D6B464]
	(No symbol) [0x00D81215]
	(No symbol) [0x00D6B216]
	(No symbol) [0x00D40D97]
	(No symbol) [0x00D4253D]
	GetHandleVerifier [0x010FABF2+2510930]
	GetHandleVerifier [0x01128EC1+2700065]
	GetHandleVerifier [0x0112C86C+2714828]
	GetHandleVerifier [0x00F33480+645344]
	(No symbol) [0x00E20FD2]
	(No symbol) [0x00E26C68]
	(No symbol) [0x00E26D4B]
	(No symbol) [0x00E30D6B]
	BaseThreadInitThunk [0x76F17D69+25]
	RtlInitializeExceptionChain [0x774EBB9B+107]
	RtlClearBits [0x774EBB1F+191]


In [91]:
PL2122 = Download(URLPL2122, 'PremierLeague', '2021-2022')
PL2223 = Download(URLPL2223, 'PremierLeague', '2022-2023')

C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\4206667366.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 30: 100%|█████████████████████████████████████████████████████████████████████████| 30/30 [03:40<00:00,  7.35s/it]
player stats summary : done


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\2573841616.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 30: 100%|█████████████████████████████████████████████████████████████████████████| 30/30 [03:30<00:00,  7.00s/it]
player stats defensive : done


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\2937695269.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


...

page 31: 100%|█████████████████████████████████████████████████████████████████████████| 31/31 [03:42<00:00,  7.17s/it]
player stats defensive : done


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\2937695269.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 31: 100%|█████████████████████████████████████████████████████████████████████████| 31/31 [03:53<00:00,  7.53s/it]
player stats offensive : done


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\167999107.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 31: 100%|█████████████████████████████████████████████████████████████████████████| 31/31 [03:29<00:00,  6.75s/it]
player stats passing : done
player stats summary saved to PremierLeague-Summary-2022-2023.csv
player stats defensive saved to PremierLeague-Defensive-2022-2023.csv
player stats offensive saved to PremierLeague-Offensive-2022-2023.csv
player stats passing saved to PremierLeague-Passing-2022-2023.csv


In [78]:
URLBD1920 = 'https://1xbet.whoscored.com/Regions/81/Tournaments/3/Seasons/7872/Stages/17682/PlayerStatistics/Germany-Bundesliga-2019-2020'
URLBD2021 = 'https://1xbet.whoscored.com/Regions/81/Tournaments/3/Seasons/8279/Stages/18762/PlayerStatistics/Germany-Bundesliga-2020-2021'
URLBD2122 = 'https://1xbet.whoscored.com/Regions/81/Tournaments/3/Seasons/8667/Stages/19862/PlayerStatistics/Germany-Bundesliga-2021-2022'
URLBD2223 = 'https://1xbet.whoscored.com/Regions/81/Tournaments/3/Seasons/9120/Stages/21026/PlayerStatistics/Germany-Bundesliga-2022-2023'

In [80]:
BD1920 = Download(URLBD1920, 'Bundesliga', '2019-2020')
BD2021 = Download(URLBD2021, 'Bundesliga','2019-2020')
BD2122 = Download(URLBD2122, 'Bundesliga', '2020-2021')
BD2223 = Download(URLBD2223, 'Bundesliga', '2022-2023')

C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\4206667366.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 27: 100%|█████████████████████████████████████████████████████████████████████████| 27/27 [07:06<00:00, 15.79s/it]
player stats summary : done


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\2573841616.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 27: 100%|█████████████████████████████████████████████████████████████████████████| 27/27 [04:28<00:00,  9.95s/it]
player stats defensive : done


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\2937695269.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


...

page 29: 100%|█████████████████████████████████████████████████████████████████████████| 29/29 [05:26<00:00, 11.24s/it]
player stats defensive : done


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\2937695269.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 29: 100%|█████████████████████████████████████████████████████████████████████████| 29/29 [06:01<00:00, 12.48s/it]
player stats offensive : done


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\167999107.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 29: 100%|█████████████████████████████████████████████████████████████████████████| 29/29 [04:28<00:00,  9.27s/it]
player stats passing : done
player stats summary saved to Bundesliga-Summary-2022-2023.csv
player stats defensive saved to Bundesliga-Defensive-2022-2023.csv
player stats offensive saved to Bundesliga-Offensive-2022-2023.csv
player stats passing saved to Bundesliga-Passing-2022-2023.csv


In [81]:
URLSA1920 = 'https://1xbet.whoscored.com/Regions/108/Tournaments/5/Seasons/7928/Stages/17835/PlayerStatistics/Italy-Serie-A-2019-2020'
URLSA2021 = 'https://1xbet.whoscored.com/Regions/108/Tournaments/5/Seasons/8330/Stages/18873/PlayerStatistics/Italy-Serie-A-2020-2021'
URLSA2122 = 'https://1xbet.whoscored.com/Regions/108/Tournaments/5/Seasons/8735/Stages/19982/PlayerStatistics/Italy-Serie-A-2021-2022'
URLSA2223 = 'https://1xbet.whoscored.com/Regions/108/Tournaments/5/Seasons/9159/Stages/21087/PlayerStatistics/Italy-Serie-A-2022-2023'

In [82]:
SA1920 = Download(URLSA1920, 'SerieA', '2019-2020')
SA2021 = Download(URLSA2021, 'SerieA', '2020-2021')
SA2122 = Download(URLSA2122, 'SerieA', '2020-2021')
SA2223 = Download(URLSA2223, 'SerieA', '2022-2023')

C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\4206667366.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 31: 100%|█████████████████████████████████████████████████████████████████████████| 31/31 [06:07<00:00, 11.87s/it]
player stats summary : done


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\2573841616.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 31: 100%|█████████████████████████████████████████████████████████████████████████| 31/31 [05:10<00:00, 10.00s/it]
player stats defensive : done


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\2937695269.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


...

page 34: 100%|█████████████████████████████████████████████████████████████████████████| 34/34 [04:42<00:00,  8.31s/it]
player stats defensive : done


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\2937695269.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 34: 100%|█████████████████████████████████████████████████████████████████████████| 34/34 [04:34<00:00,  8.07s/it]
player stats offensive : done


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\167999107.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 34: 100%|█████████████████████████████████████████████████████████████████████████| 34/34 [04:29<00:00,  7.92s/it]
player stats passing : done
player stats summary saved to SerieA-Summary-2022-2023.csv
player stats defensive saved to SerieA-Defensive-2022-2023.csv
player stats offensive saved to SerieA-Offensive-2022-2023.csv
player stats passing saved to SerieA-Passing-2022-2023.csv


In [83]:
URLLONE1920 = 'https://1xbet.whoscored.com/Regions/74/Tournaments/22/Seasons/7814/Stages/17593/PlayerStatistics/France-Ligue-1-2019-2020'
URLLONE2021 = 'https://1xbet.whoscored.com/Regions/74/Tournaments/22/Seasons/8185/Stages/18594/PlayerStatistics/France-Ligue-1-2020-2021'
URLLONE2122 = 'https://1xbet.whoscored.com/Regions/74/Tournaments/22/Seasons/8671/Stages/19866/PlayerStatistics/France-Ligue-1-2021-2022'
URLLONE2223 = 'https://1xbet.whoscored.com/Regions/74/Tournaments/22/Seasons/9129/Stages/21037/PlayerStatistics/France-Ligue-1-2022-2023'

In [84]:
LONE1920 = Download(URLLONE1920, 'Ligue1', '2019-2020')
LONE2021 = Download(URLLONE2021, 'Ligue1', '2019-2020')
LONE2122 = Download(URLLONE2122, 'Ligue1', '2020-2021')
LONE2223 = Download(URLLONE2223, 'Ligue1', '2022-2023')

C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\4206667366.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 31: 100%|█████████████████████████████████████████████████████████████████████████| 31/31 [04:02<00:00,  7.81s/it]
player stats summary : done


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\2573841616.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 31: 100%|█████████████████████████████████████████████████████████████████████████| 31/31 [03:48<00:00,  7.36s/it]
player stats defensive : done


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\2937695269.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


...

page 32: 100%|█████████████████████████████████████████████████████████████████████████| 32/32 [03:44<00:00,  7.01s/it]
player stats defensive : done


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\2937695269.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 32: 100%|█████████████████████████████████████████████████████████████████████████| 32/32 [04:08<00:00,  7.77s/it]
player stats offensive : done


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\167999107.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 32: 100%|█████████████████████████████████████████████████████████████████████████| 32/32 [03:43<00:00,  6.97s/it]
player stats passing : done
player stats summary saved to Ligue1-Summary-2022-2023.csv
player stats defensive saved to Ligue1-Defensive-2022-2023.csv
player stats offensive saved to Ligue1-Offensive-2022-2023.csv
player stats passing saved to Ligue1-Passing-2022-2023.csv


In [85]:
URLLIGA1920 = 'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/8618/Stages/12496/PlayerStatistics/England-Premier-League-2019-2020'
URLLIGA2021 = 'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/8228/Stages/18685/PlayerStatistics/England-Premier-League-2020-2021'
URLLIGA2122 = 'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/8618/Stages/19793/PlayerStatistics/England-Premier-League-2021-2022'
URLLIGA2223 = 'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/9075/Stages/20934/PlayerStatistics/England-Premier-League-2022-2023'

In [86]:
LIGA1920 = Download(URLLIGA1920, 'LaLiga','2019-2020')
LIGA2021 = Download(URLLIGA2021, 'LaLiga','2019-2020')
LIGA2122 = Download(URLLIGA2122, 'LaLiga', '2020-2021')
LIGA2223 = Download(URLLIGA2223, 'LaLiga', '2022-2023')

C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\4206667366.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 30: 100%|█████████████████████████████████████████████████████████████████████████| 30/30 [04:05<00:00,  8.19s/it]
player stats summary : done


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\2573841616.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 30: 100%|█████████████████████████████████████████████████████████████████████████| 30/30 [03:36<00:00,  7.23s/it]
player stats defensive : done


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\2937695269.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


...

page 31: 100%|█████████████████████████████████████████████████████████████████████████| 31/31 [04:02<00:00,  7.82s/it]
player stats defensive : done


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\2937695269.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 31: 100%|█████████████████████████████████████████████████████████████████████████| 31/31 [03:54<00:00,  7.57s/it]
player stats offensive : done


C:\Users\lucas\AppData\Local\Temp\ipykernel_75360\167999107.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


page 31: 100%|█████████████████████████████████████████████████████████████████████████| 31/31 [03:20<00:00,  6.47s/it]
player stats passing : done
player stats summary saved to LaLiga-Summary-2022-2023.csv
player stats defensive saved to LaLiga-Defensive-2022-2023.csv
player stats offensive saved to LaLiga-Offensive-2022-2023.csv
player stats passing saved to LaLiga-Passing-2022-2023.csv


In [67]:
import pandas as pd
df = pd.read_csv('PremierLeague-Defensive-2019-2020.csv')

In [68]:
df.head()

,Name,Team,Age,Position,Apps,Minutes,Tackles per game,Interceptions per game,Fouls per game,Offsides per game,Clearances per game,Dribbled past per game,Outfield blocks per game,Own goals,Rating
0,Harry Kane,Tottenham,29,"AM(C),FW",35,3087,0.6,0.3,0.7,0.0,1.1,0.5,0.3,0,7.79
1,Kevin De Bruyne,Man City,31,"M(CLR),FW",23(2),2001,1.4,0.4,1.1,0.0,0.3,1.4,0.0,0,7.65
2,Jack Grealish,Aston Villa,27,"AM(CL),FW",24(2),2184,1.2,0.5,1.5,0.0,0.2,0.3,0.1,0,7.56
3,Bruno Fernandes,Man Utd,28,M(CLR),35(2),3110,1.5,0.7,1.2,0.0,0.7,2.0,0.0,0,7.43
4,Tomás Soucek,West Ham,28,DMC,38,3420,2.1,1.6,1.8,0.1,2.7,1.3,0.5,1,7.36
